# Part I. ETL Pipeline for Pre-Processing the Files

In [115]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from tabulate import tabulate



In [116]:
# Creating list of filepaths to process original event csv data files

# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


In [117]:
# Processing the files to create the data file csv that will be used for Apache Casssandratables

# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# to get total number of rows 
#print(len(full_data_rows_list))
# to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [118]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [119]:
# Creating a Cluster
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

In [120]:
# Create Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS projectone 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

In [121]:
# Set Keyspace
try:
    session.set_keyspace('projectone')
except Exception as e:
    print(e)

In [122]:
# Query1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

# Primary Key: (sessionId, itemInSession)
# Partition Key (sessionId): Chosen because the query filters by sessionId. 
# This ensures all data for a session is stored together, optimizing for query efficiency.
# Clustering Column (itemInSession): Used to further refine data within a session. 
# It allows the database to efficiently locate the specific item in a session, 
# which is essential for the query. This design effectively supports the query requirement to retrieve 
# specific song details based on sessionId and itemInSession.


query1 = """
CREATE TABLE IF NOT EXISTS session_songs (
    sessionId int,
    itemInSession int,
    artist text,
    song text,
    length float,
    PRIMARY KEY (sessionId, itemInSession)
);
"""
session.execute(query1)


                    

In [123]:
# Insert Data

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query1 = "INSERT INTO session_songs (sessionId, itemInSession, artist, song, length) "
        query1 = query1 + "VALUES (%s, %s, %s, %s, %s)"
        
        session.execute(query1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
            


In [124]:
# Verify Data Insertion

query1 = "SELECT artist, song, length FROM session_songs WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query1)
    # Convert query results to a DataFrame
    df = pd.DataFrame(list(rows))
    # Display the DataFrame using tabulate
    print(tabulate(df, headers='keys', tablefmt='fancy_grid'))
except Exception as e:
    print(e)

╒════╤═══════════╤═════════════════════════════════╤══════════╕
│    │ artist    │ song                            │   length │
╞════╪═══════════╪═════════════════════════════════╪══════════╡
│  0 │ Faithless │ Music Matters (Mark Knight Dub) │  495.307 │
╘════╧═══════════╧═════════════════════════════════╧══════════╛


In [125]:
# Query2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


# Primary Key: Composite key ((userId, sessionId), itemInSession)
# Composite Partition Key (userId, sessionId): 
# Enables efficient queries by grouping all data for a specific user and session together.
# Clustering Column (itemInSession): 
# Sorts songs within each user-session pair in the order they were played.

query2 = """
CREATE TABLE IF NOT EXISTS user_session_songs (
    userId int,
    sessionId int,
    itemInSession int,
    artist text,
    song text,
    firstName text,
    lastName text,
    PRIMARY KEY ((userId, sessionId), itemInSession)
) WITH CLUSTERING ORDER BY (itemInSession ASC);
"""
session.execute(query2)

                    

In [126]:
# Insert Data

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query2 = "INSERT INTO user_session_songs (userId, sessionId, itemInSession, artist, song, firstName, lastName) "
        query2 = query2 + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

                    

In [127]:
# Verify Data Insertion

query2 = "SELECT artist, song, firstname, lastname FROM user_session_songs WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query2)
    # Convert query results to a DataFrame
    df = pd.DataFrame(list(rows))
    # Display the DataFrame using tabulate
    print(tabulate(df, headers='keys', tablefmt='fancy_grid'))
except Exception as e:
    print(e)


╒════╤═══════════════════╤══════════════════════════════════════════════════════╤═════════════╤════════════╕
│    │ artist            │ song                                                 │ firstname   │ lastname   │
╞════╪═══════════════════╪══════════════════════════════════════════════════════╪═════════════╪════════════╡
│  0 │ Down To The Bone  │ Keep On Keepin' On                                   │ Sylvie      │ Cruz       │
├────┼───────────────────┼──────────────────────────────────────────────────────┼─────────────┼────────────┤
│  1 │ Three Drives      │ Greece 2000                                          │ Sylvie      │ Cruz       │
├────┼───────────────────┼──────────────────────────────────────────────────────┼─────────────┼────────────┤
│  2 │ Sebastien Tellier │ Kilometer                                            │ Sylvie      │ Cruz       │
├────┼───────────────────┼──────────────────────────────────────────────────────┼─────────────┼────────────┤
│  3 │ Lonnie Gordo

In [128]:
#Query3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


# Primary Key: (song, userId)
# Partition Key (song): 
# Facilitates efficient queries by song title, as the query filters on this.
# Clustering Column (userId): 
# Ensures unique entries for users per song and provides sorted order of users for each song.

query3 = """
CREATE TABLE IF NOT EXISTS song_listeners (
    song text,
    userId int,
    firstname text,
    lastname text,
    PRIMARY KEY (song, userId)
);
"""
session.execute(query3)



In [129]:
# Insert Data

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query3 = "INSERT INTO song_listeners (song, userId, firstname, lastname) "
        query3 = query3 + "VALUES (%s, %s, %s, %s)"
        session.execute(query3, (line[9], int(line[10]), line[1], line[4]))


In [130]:
# Verify Data Insertion

query3 = "SELECT firstname, lastname FROM song_listeners WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query3)
    # Convert query results to a DataFrame
    df = pd.DataFrame(list(rows))
    # Display the DataFrame using tabulate
    print(tabulate(df, headers='keys', tablefmt='fancy_grid'))
except Exception as e:
    print(e)



╒════╤═════════════╤════════════╕
│    │ firstname   │ lastname   │
╞════╪═════════════╪════════════╡
│  0 │ Jacqueline  │ Lynch      │
├────┼─────────────┼────────────┤
│  1 │ Tegan       │ Levine     │
├────┼─────────────┼────────────┤
│  2 │ Sara        │ Johnson    │
╘════╧═════════════╧════════════╛


In [131]:
# List of tables to be dropped before closing out the sessions
tables_to_drop = ['session_songs', 'user_session_songs', 'song_listeners']

# Looping through the list and dropping each table
for table in tables_to_drop:
    query = f"DROP TABLE IF EXISTS {table}"
    try:
        session.execute(query)
        print(f"Table '{table}' dropped successfully.")
    except Exception as e:
        print(f"Error dropping table {table}: {e}")



Table 'session_songs' dropped successfully.
Table 'user_session_songs' dropped successfully.
Table 'song_listeners' dropped successfully.


### Close the session and cluster connection¶

In [132]:
#Close the session and cluster connection

session.shutdown()
cluster.shutdown()